In [ ]:
using DifferentialEquations
using PyPlot
using ReactionMechanismSimulator

In [ ]:
phaseDict = readinput("../src/testing/superminimal.rms") #load mechanism dictionary

In [ ]:
spcs = phaseDict["gas"]["Species"]; #mechanism dictionaries index:  phaseDict[phasename]["Species" or "Reactions"]
rxns = phaseDict["gas"]["Reactions"];

In [ ]:
ig = IdealGas(spcs,rxns,name="gas"); #Define the phase (how species thermodynamic and kinetic properties calculated)
initialconds = Dict(["T"=>1000.0,"P"=>1e5,"H2"=>0.67,"O2"=>0.33]); #Set simulation Initial Temp and Pressure
domain,y0 = ConstantTPDomain(phase=ig,initialconds=initialconds;sensitivity=true); #Define the domain (encodes how system thermodynamic properties calculated)

In [ ]:
react = Reactor(domain,y0,(0.0,150.1)); #Create the reactor object

In [ ]:
sol = solve(react.ode,CVODE_BDF(),abstol=1e-20,reltol=1e-12); #solve the ode associated with the reactor

In [ ]:
bsol = Simulation(sol,domain);

In [ ]:
plotmolefractions(bsol,150.0; t0=1e-15, N=1000, tol=0.01)

In [ ]:
plotmaxratesensitivity(bsol,"H2")

In [ ]:
plotmaxthermosensitivity(bsol,"H2")

In [ ]:
plotrops(bsol,"OH(D)",tol=0.01)